In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [86]:
import yaml
import random
import json
from collections import defaultdict
from probis import *
from run import *

In [3]:
with open("../cfg.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [80]:
uniprot2recs,\
uniprot2ligs,\
uniprot2pockets,\
pocket2uniprots,\
pocket2recs,\
pocket2ligs = get_uniprot_dicts(cfg)
pocket2recs,\
pocket2ligs,\
ligfile2lig = save_all_structures(cfg)
pocket2rep_rec = get_rep_recs(cfg)
full_scores = convert_inter_results_to_json(cfg)
struct_df = filter_struct_df(cfg)
activities_filtered = add_sdfs_to_activities(cfg)
final_uniprots, final_pockets = filter_uniprots(cfg)

rec2pocketfile, rec2res_num = save_all_pockets(cfg)
pocket_centers, pocket_sizes = get_all_pocket_bounds(cfg)

Using cached data from get_uniprot_dicts
Using cached data from save_all_structures
Using cached data from get_rep_recs
Using cached data from convert_inter_results_to_json
Using cached data from filter_struct_df
Using cached data from add_sdfs_to_activities
Using cached data from filter_uniprots
Using cached data from save_all_pockets
Using cached data from get_all_pocket_bounds


In [62]:
struct_df

,lig_smiles,lig_file,lig_pdb,pocket,ex_rec_file,ex_rec_pdb,ex_rec_pocket_file,num_pocket_residues,pocket_center_x,pocket_center_y,pocket_center_z,pocket_size_x,pocket_size_y,pocket_size_z
0,CCCCCCCCCCCCCCCCCCCC(=O)NCCO,FABP5_HUMAN_2_135_0/4azr_a9m_lig.sdf,4azr,FABP5_HUMAN_2_135_0,FABP5_HUMAN_2_135_0/5hz5_A_rec.pdb,5hz5,FABP5_HUMAN_2_135_0/5hz5_A_rec_pocket.pdb,48,22.453499,4.836500,-1.527500,32.376998,24.219000,21.589000
1,CCc1c(-c2ccccc2)c(-c2ccccc2)nn1-c1ccccc1-c1ccc...,FABP5_HUMAN_2_135_0/4azm_t4b_lig.sdf,4azm,FABP5_HUMAN_2_135_0,FABP5_HUMAN_2_135_0/5hz5_A_rec.pdb,5hz5,FABP5_HUMAN_2_135_0/5hz5_A_rec_pocket.pdb,48,22.453499,4.836500,-1.527500,32.376998,24.219000,21.589000
2,CCCCC/C=C\C/C=C\CCCCCCCC(=O)O,FABP5_HUMAN_2_135_0/4lkt_eic_lig.sdf,4lkt,FABP5_HUMAN_2_135_0,FABP5_HUMAN_2_135_0/5ur9_A_rec.pdb,5ur9,FABP5_HUMAN_2_135_0/5ur9_A_rec_pocket.pdb,47,22.453499,4.836500,-1.527500,32.376998,24.219000,21.589000
3,CCCCCCCCCCCCCCCC(=O)O,FABP5_HUMAN_2_135_0/1b56_plm_lig.sdf,1b56,FABP5_HUMAN_2_135_0,FABP5_HUMAN_2_135_0/4azr_A_rec.pdb,4azr,FABP5_HUMAN_2_135_0/4azr_A_rec_pocket.pdb,50,22.453499,4.836500,-1.527500,32.376998,24.219000,21.589000
4,CCCCCCCCCCCCCC(=O)O,FABP5_HUMAN_2_135_0/5ur9_myr_lig.sdf,5ur9,FABP5_HUMAN_2_135_0,FABP5_HUMAN_2_135_0/4azm_A_rec.pdb,4azm,FABP5_HUMAN_2_135_0/4azm_A_rec_pocket.pdb,35,22.453499,4.836500,-1.527500,32.376998,24.219000,21.589000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16403,CC/C=C\C[C@H]1C(=O)CC[C@@H]1CC(=O)O,JAR1_ARATH_1_575_0/5eci_jaa_lig.sdf,5eci,JAR1_ARATH_1_575_0,JAR1_ARATH_1_575_0/5gzz_A_rec.pdb,5gzz,JAR1_ARATH_1_575_0/5gzz_A_rec_pocket.pdb,47,47.392500,32.963500,87.952503,28.465000,22.447001,21.887001
16404,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO[P@@](=O)(O)O[P@...,JAR1_ARATH_1_575_0/5eci_atp_lig.sdf,5eci,JAR1_ARATH_1_575_0,JAR1_ARATH_1_575_0/4epl_A_rec.pdb,4epl,JAR1_ARATH_1_575_0/4epl_A_rec_pocket.pdb,44,47.392500,32.963500,87.952503,28.465000,22.447001,21.887001
16405,CC/C=C\C[C@H]1C(=O)CC[C@@H]1CC(=O)O,JAR1_ARATH_1_575_0/5ecl_jaa_lig.sdf,5ecl,JAR1_ARATH_1_575_0,JAR1_ARATH_1_575_0/5gzz_A_rec.pdb,5gzz,JAR1_ARATH_1_575_0/5gzz_A_rec_pocket.pdb,47,47.392500,32.963500,87.952503,28.465000,22.447001,21.887001
16406,CCCCCCCCO[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O)[C@...,FZD5_HUMAN_27_158_0/5urz_bog_lig.sdf,5urz,FZD5_HUMAN_27_158_0,FZD5_HUMAN_27_158_0/5ury_A_rec.pdb,5ury,FZD5_HUMAN_27_158_0/5ury_A_rec_pocket.pdb,14,-37.898998,62.783998,11.589000,20.320000,20.221996,27.168000


In [87]:
activities = make_final_activities_df(cfg,\
                         activities_filtered,\
                         uniprot2pockets,\
                         pocket2recs,\
                             rec2pocketfile,\
                             rec2res_num,\
                             pocket_centers,\
                             pocket_sizes)

Running make_final_activities_df


In [88]:
def save_clustered_activities(cfg, activities, splits):
    for split, pockets in splits.items():
        split_act = activities.query("pocket in @pockets").reset_index(drop=True)
        split_act.to_csv(cfg["bigbind_folder"] + f"/activities_{split}.csv", index=False)
save_clustered_activities(cfg, activities, splits)